In [30]:
import requests
import time
import hashlib
from neo4j import GraphDatabase
import json

In [41]:
API_PUBLIC_KEY = "929b4b43b3ab59b27417573fcd5868b9"
API_PRIVATE_KEY = "27bc1b826da0c2def3525f69a2668e7834744938"
API_LIMIT = 10
API_OFFSET = 0

In [28]:
current_timestamp = time.time()
hash = hashlib.md5((str(current_timestamp)+API_PRIVATE_KEY+API_PUBLIC_KEY).encode())

In [29]:
response_API = requests.get('https://gateway.marvel.com:443/v1/public/characters?limit=%s&offset=%s&apikey=929b4b43b3ab59b27417573fcd5868b9&hash=%s&ts=%s'%(API_LIMIT,API_OFFSET,hash.hexdigest(),str(current_timestamp)))
with open("test_chiamata.json","w") as out_file:
  out_file.write(response_API.text)


In [6]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j","1234qwer"))

In [14]:
def add_test_node(tx, name, label):
  query_string = 'CREATE (%s:%s{name:"%s"})'%(name, label, name)
  print(query_string)
  tx.run(query_string)

In [46]:
import logging
neo4j_log = logging.getLogger("neo4j.bolt")
neo4j_log.setLevel(logging.WARNING)
def add_character_node(tx, name):
  query_string = 'CREATE (n:character{name:"%s"})'%( name)
  print(query_string)
  tx.run(query_string)

In [15]:
with driver.session() as session:
  session.write_transaction(add_test_node, "NodoCodice","Codice")

CREATE (NodoCodice:Codice{name:"NodoCodice"})


In [26]:

response_JSON = json.loads(response_API.text)
for character in response_JSON['data']['results']:
  print(character['name'])
  with driver.session() as session:
    session.write_transaction(add_character_node, character['name'])

3-D Man
CREATE (n:character{name:"3-D Man"})
A-Bomb (HAS)
CREATE (n:character{name:"A-Bomb (HAS)"})
A.I.M.
CREATE (n:character{name:"A.I.M."})
Aaron Stack
CREATE (n:character{name:"Aaron Stack"})
Abomination (Emil Blonsky)
CREATE (n:character{name:"Abomination (Emil Blonsky)"})
Abomination (Ultimate)
CREATE (n:character{name:"Abomination (Ultimate)"})
Absorbing Man
CREATE (n:character{name:"Absorbing Man"})
Abyss
CREATE (n:character{name:"Abyss"})
Abyss (Age of Apocalypse)
CREATE (n:character{name:"Abyss (Age of Apocalypse)"})
Adam Destine
CREATE (n:character{name:"Adam Destine"})


In [47]:
results_ended = False
iteration_number = 0
API_OFFSET = 0
while not results_ended and iteration_number < 50:
  print("********Starting iteration n. %s \n" %(str(iteration_number)))
  response_API = requests.get('https://gateway.marvel.com:443/v1/public/characters?limit=%s&offset=%s&apikey=929b4b43b3ab59b27417573fcd5868b9&hash=%s&ts=%s'%(API_LIMIT,API_OFFSET,hash.hexdigest(),str(current_timestamp)))
  response_JSON = json.loads(response_API.text)
  try:
    if len(response_JSON['data']['results']) != 0:
      for character in response_JSON['data']['results']:
        with driver.session() as session:
          
            session.write_transaction(add_character_node, character['name'])
          
      API_OFFSET += API_LIMIT
      print("********Ending iteration n. %s \n" %(str(iteration_number)))
      iteration_number +=1
    else:
      results_ended = True
  except Exception:
          print(Exception)


********Starting iteration n. 0 

CREATE (n:character{name:"3-D Man"})
CREATE (n:character{name:"A-Bomb (HAS)"})
CREATE (n:character{name:"A.I.M."})
CREATE (n:character{name:"Aaron Stack"})
CREATE (n:character{name:"Abomination (Emil Blonsky)"})
CREATE (n:character{name:"Abomination (Ultimate)"})
CREATE (n:character{name:"Absorbing Man"})
CREATE (n:character{name:"Abyss"})
CREATE (n:character{name:"Abyss (Age of Apocalypse)"})
CREATE (n:character{name:"Adam Destine"})
********Ending iteration n. 0 

********Starting iteration n. 1 

CREATE (n:character{name:"Adam Warlock"})
CREATE (n:character{name:"Aegis (Trey Rollins)"})
CREATE (n:character{name:"Aero (Aero)"})
CREATE (n:character{name:"Agatha Harkness"})
CREATE (n:character{name:"Agent Brand"})
CREATE (n:character{name:"Agent X (Nijo)"})
CREATE (n:character{name:"Agent Zero"})
CREATE (n:character{name:"Agents of Atlas"})
CREATE (n:character{name:"Aginar"})
CREATE (n:character{name:"Air-Walker (Gabriel Lan)"})
********Ending iteratio

KeyboardInterrupt: 